[https://peps.python.org/pep-0380/](https://peps.python.org/pep-0380/)

```python
RESULT = yield from EXPR

# is semantically equivalent to
def yield_from(EXPR):
    _i = iter(EXPR)
    try:
        _y = next(_i)
    except StopIteration as _e:
        _r = _e.value
    else:
        while 1:
            try:
                _s = yield _y
            except GeneratorExit as _e:
                try:
                    _m = _i.close
                except AttributeError:
                    pass
                else:
                    _m()
                raise _e
            except BaseException as _e:
                _x = sys.exc_info()
                try:
                    _m = _i.throw
                except AttributeError:
                    raise _e
                else:
                    try:
                        _y = _m(*_x)
                    except StopIteration as _e:
                        _r = _e.value
                        break
            else:
                try:
                    if _s is None:
                        _y = next(_i)
                    else:
                        _y = _i.send(_s)
                except StopIteration as _e:
                    _r = _e.value
                    break
    RESULT = _r

```

> 可以看出，yield from 后面的表达式需要是一个可迭代对象


## 从yield到yield from

### 基础嵌套任务

In [25]:
def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = big_step()
    print(f"    end big_step {big_result}")
    print("end task")

def big_step():
    print(f"        begin small_step")
    small_result = small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000

def small_step():
    print("         努力工作")
    return 123

In [4]:
one_task()

begin task
     begin big_step
        begin small_step
         努力工作
        end small_step with 123
    end big_step 123000
end task


In [7]:
# time.sleep 模拟耗时阻塞
import time

def small_step():
    print("         休息一下")
    time.sleep(2)
    print("         努力工作")
    return 123

one_task()

begin task
     begin big_step
        begin small_step
         休息一下
         努力工作
        end small_step with 123
    end big_step 123000
end task


### 任务改写01

> small_step加上yield

In [26]:
def small_step():
    print("         休息一下")
    yield time.sleep(2)
    print("         努力工作")
    return 123

one_task()

begin task
     begin big_step
        begin small_step
        end small_step with <generator object small_step at 0x10da6a510>


TypeError: unsupported operand type(s) for *: 'generator' and 'int'

> 运行失败，因为yield具有传染性

### 任务改写02

> big_step接收small_step的值

In [22]:
def big_step():
    print(f"        begin small_step")
    small_coro = small_step()  # 接收返回的生成器
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            small_result = e.value
            break
        else:
            pass

    print(f"        end small_step with {small_result}")
    return small_result * 1000

one_task()


<generator object one_task at 0x10da6a120>

### 任务改写03

> yield改为yield from

In [30]:
def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000

one_task()

begin task
     begin big_step
    end big_step <generator object big_step at 0x10da6a2e0>
end task


In [31]:
def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")

one_task()

<generator object one_task at 0x10da6add0>

In [32]:
class Task:
    def __init__(self, coro):
        self.coro = coro

    def run(self):
        print("-----")
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                result = e.value
                break
            else:
                # x?
                result = x
        print(f"----- {result}")


t = Task(one_task())
t.run()

-----
begin task
     begin big_step
        begin small_step
         休息一下
         努力工作
        end small_step with 123
    end big_step 123000
end task
----- None
